In [109]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os
import PIL
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential 
import pandas as pd
import numpy as np
import pathlib

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, TimeDistributed, Conv2D, MaxPooling2D, Flatten, Reshape
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dropout
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.callbacks import EarlyStopping
import datetime
from tensorflow.keras.callbacks import Callback

This is demo code on to check how preprocess the images for vgg network

In [110]:


class Manage_data():
    def __init__(self):
        data_dir='/home/rag-tt/workspace/tactile_images/'
        self.data_dir= pathlib.Path(data_dir)

    def find_no_of_images(self, obj_id):
        image_dir = os.path.join(self.data_dir, str(obj_id))
        image_dir= pathlib.Path(image_dir)
        no_of_images= len(list(image_dir.glob('*.jpg')))
        return no_of_images
    
    

    def parse_function_sequential(self, filenames, label):
        images = []
        for filename in filenames:
            image_string = tf.io.read_file(filename)
            image_decoded = tf.image.decode_jpeg(image_string, channels=3)
            image_resized = tf.image.resize(image_decoded, [480, 640])  # Adjust size as needed
            # Ensure images are float32 and normalized between 0 and 1
            images.append(image_resized)
        images = tf.stack(images)
        return images, label
    
    def parse_function_vgg(self, filenames, label):
        images = []
        for filename in filenames:
            image_string = tf.io.read_file(filename)
            image_decoded = tf.image.decode_jpeg(image_string, channels=3)
            image_resized = tf.image.resize(image_decoded, [224, 224])  # Adjust size as needed
            # Convert image to a float32 tensor and preprocess it for VGG16
            image = tf.cast(image_resized, tf.float32)
            image = preprocess_input(image)
            # Ensure images are float32 and normalized between 0 and 1
            images.append(image)
        images = tf.stack(images)
        return images, label
    
    def load_sequential_data(self, no_of_samples = 600):
        file_paths = []
        image_paths = []
        sequential_image_paths = []
        y = []
        window_size = 5
        for obj_id in range(no_of_samples):
            no_of_images = self.find_no_of_images(obj_id)
            if no_of_images < 40:
                continue
            
            csv_path = os.path.join(self.data_dir, str(obj_id),'slip_log.csv')
            label = np.genfromtxt(csv_path, delimiter=',', skip_header=1, usecols=1, dtype=None, encoding=None)
            y.append(label[:-4])
            
            for img_id in range(no_of_images):
                image_path = os.path.join(self.data_dir, str(obj_id), str(img_id)+ '.jpg')
                image_paths.append(image_path)
            for i in range(0, len(image_paths) - 4):  # Ensuring sequences of 5 images
                row = image_paths[i:i+5]
                sequential_image_paths.append(row)
            image_paths = []

        y = np.concatenate(y)
        y = np.array(y)

        file_paths = np.array(sequential_image_paths)

        return file_paths, y
    
    def create_sequential_dataset(self,file_paths, labels):
                # Create a TensorFlow dataset from the file paths and labels
        dataset = tf.data.Dataset.from_tensor_slices((file_paths, labels))
        
        # # Map the function to each sequence of file paths and labels
        # dataset = dataset.map(
        #     lambda file_paths, label: self.parse_function_sequential(file_paths, label),
        #     num_parallel_calls=tf.data.AUTOTUNE
        # )
        def wrapped_parse_function(filenames, label):
            images, label = tf.py_function(func=self.parse_function_vgg, inp=[filenames, label], Tout=[tf.float32, tf.int64])
            images.set_shape((5, 224, 224, 3))  # Explicitly set the shape
            label.set_shape([])  # Explicitly set the shape for the label
            return images, label
        # # Map the parse_function to the dataset using tf.py_function
        # dataset = dataset.map(lambda file_paths, label: tf.py_function(func=self.parse_function_sequential, inp=[file_paths, label], Tout=[tf.float32, tf.int64]),
        #               num_parallel_calls=tf.data.AUTOTUNE)
        
        dataset = dataset.map(wrapped_parse_function, num_parallel_calls=tf.data.AUTOTUNE)
        #     # Use tf.py_function to apply the parse_function
        # def wrapped_parse_function(filenames, label):
        #     return tf.py_function(func=self.parse_function_sequential, inp=[filenames, label], Tout=[tf.float32, tf.float32])

        # dataset = dataset.map(wrapped_parse_function, num_parallel_calls=tf.data.AUTOTUNE)

        dataset = dataset.batch(32)  # Adjust batch size as needed
        self.dataset = dataset.prefetch(buffer_size=tf.data.AUTOTUNE)


    def split_dataset(self, file_paths):
        dataset_size = len(file_paths)
        train_size = int(0.8 * dataset_size)
        validation_size = int(0.2 * dataset_size)
        print('datasize', dataset_size)
        print('train size',train_size)
        print('validation size', validation_size)
        self.train_dataset = self.dataset.take(train_size)
        self.val_dataset = self.dataset.skip(train_size)
    
    # Function to count the elements
    def count_elements(self,dataset):
        return dataset.reduce(0, lambda x, _: x + 1).numpy()
    
    def count_elements2(self,dataset):
        count = 0
        for _ in dataset:
            count += 1
        return count



In [111]:
class Manage_data2():
    def __init__(self):
        data_dir='/home/rag-tt/workspace/tactile_images/'
        self.data_dir= pathlib.Path(data_dir)
        
    def find_no_of_images(self, obj_id):
        image_dir = os.path.join(self.data_dir, str(obj_id))
        image_dir= pathlib.Path(image_dir)
        no_of_images= len(list(image_dir.glob('*.jpg')))
        return no_of_images
    
    def load_data(self, no_of_samples = 600):
        file_paths = []
        image_paths = []
        sequential_image_paths = []
        y = []
        window_size = tune.sequence_of_image
        for obj_id in range(no_of_samples):
            no_of_images = self.find_no_of_images(obj_id)
            if no_of_images < 40:
                continue
            
            csv_path = os.path.join(self.data_dir, str(obj_id),'slip_log.csv')
            label = np.genfromtxt(csv_path, delimiter=',', skip_header=1, usecols=1, dtype=None, encoding=None)
            y.append(label[:-(tune.sequence_of_image-1)])
            
            for img_id in range(no_of_images):
                image_path = os.path.join(self.data_dir, str(obj_id), str(img_id)+ '.jpg')
                image_paths.append(image_path)
            for i in range(0, len(image_paths) - (tune.sequence_of_image-1)):  # Ensuring sequences of 5 images
                row = image_paths[i:i+tune.sequence_of_image]
                sequential_image_paths.append(row)
            image_paths = []

        y = np.concatenate(y)
        self.labels = np.array(y)

        self.file_paths = np.array(sequential_image_paths)
        
    def shuffle_file_paths(self):
        # Shuffle the dataset
        indices = np.arange(len(self.file_paths))
        np.random.shuffle(indices)
        self.file_paths = self.file_paths[indices]
        self.labels = self.labels[indices]

    def create_split_filepaths(self,train=0.7,val=0.2):
        dataset_size = len(self.file_paths)
        train_size = int(train * dataset_size)

        val_size = int(val * dataset_size)
        
        test_size = dataset_size - train_size - val_size
        
        
        self.train_filepaths = self.file_paths[ : train_size]
        self.val_filepaths = self.file_paths[train_size : train_size+val_size]
        self.test_filepaths = self.file_paths[train_size+val_size : ]
        
        self.train_labels = self.labels[ : train_size]
        self.val_labels = self.labels[train_size : train_size+val_size]
        self.test_labels = self.labels[train_size+val_size : ]
        
        # Check the sizes of the splits
        assert len(self.train_filepaths) == train_size, "Training set size mismatch"
        assert len(self.val_filepaths) == val_size, "Validation set size mismatch"
        assert len(self.test_filepaths) == test_size, "Test set size mismatch"
        assert len(self.train_labels) == train_size, "Training set size mismatch"
        assert len(self.val_labels) == val_size, "Validation set size mismatch"
        assert len(self.test_labels) == test_size, "Test set size mismatch"
        
    def parse_function_vgg(self, filenames, label):
        images = []
        for filename in filenames:
            image_string = tf.io.read_file(filename)
            image_decoded = tf.image.decode_jpeg(image_string, channels=3)
            image_resized = tf.image.resize(image_decoded, [224, 224])  # Adjust size as needed
            # Convert image to a float32 tensor and preprocess it for VGG16
            image = tf.cast(image_resized, tf.float32)
            image = preprocess_input(image)
            # Ensure images are float32 and normalized between 0 and 1
            images.append(image)
        images = tf.stack(images)
        return images, label
        
    def create_dataset(self,file_paths, labels):
                # Create a TensorFlow dataset from the file paths and labels
        dataset = tf.data.Dataset.from_tensor_slices((file_paths, labels))
        

        def wrapped_parse_function(filenames, label):
            images, label = tf.py_function(func=self.parse_function_vgg, inp=[filenames, label], Tout=[tf.float32, tf.int64])
            images.set_shape((5, 224, 224, 3))  # Explicitly set the shape
            label.set_shape([])  # Explicitly set the shape for the label
            return images, label
 
        
        dataset = dataset.map(wrapped_parse_function, num_parallel_calls=tf.data.AUTOTUNE)

        dataset = dataset.batch(8)  # Adjust batch size as needed
        dataset = dataset.prefetch(buffer_size=tf.data.AUTOTUNE)
        return dataset
        
    def create_split_datasets(self):
        self.train_dataset = self.create_dataset(self.train_filepaths, self.train_labels)
        self.val_dataset = self.create_dataset(self.val_filepaths, self.val_labels)
        self.test_dataset = self.create_dataset(self.test_filepaths, self.test_labels)

In [112]:
# Import the VGG16 model with pre-trained weights
# vgg_model = tf.keras.applications.VGG16(weights='imagenet', include_top=False)
# vgg_model = tf.keras.applications.VGG16(weights='imagenet')
# Print the summary of the VGG16 model
# vgg_model.summary()

In [113]:
# vgg_model.input_shape

In [114]:

class create_network():

    def __init__(self):
        self.x =0
        
    def cnn_lstm1(self):

        # Define CNN model
        cnn_model = Sequential([
            Conv2D(32, (3, 3), activation='relu', input_shape=(480, 640, 3),kernel_regularizer=l1(tune.regularizaion_const)),
            MaxPooling2D((2, 2)),
            Conv2D(32, (3, 3), activation='relu'),
            MaxPooling2D((2, 2)),
            Conv2D(32, (3, 3), activation='relu'),
            MaxPooling2D((2, 2)),
            Flatten()  # Flatten the spatial dimensions
        ])

        
        # Define LSTM model
        lstm_model = Sequential([
            LSTM(64,input_shape=(tune.sequence_of_image, 144768),kernel_regularizer=l1(tune.regularizaion_const) ),
            Dense(8, activation='relu', kernel_regularizer=l1(tune.regularizaion_const)),
            Dense(1, activation='sigmoid'),
        ])

        # Combine CNN and LSTM models
        self.model = Sequential([
            TimeDistributed(cnn_model, input_shape=(tune.sequence_of_image, 480, 640, 3)),  # Apply CNN to each frame in the sequence
            (Reshape((5,144768))),
            lstm_model,
        ])
        self.model.summary()

    def vgg_lstm(self):
        # VGG16 model with pre-trained weights
        #include top  false remove the final classification layer
        vgg_model = tf.keras.applications.VGG16(weights='imagenet',include_top=False, input_shape=(224, 224, 3))
        # Freeze the VGG16 layers if you don't want to train them
        for layer in vgg_model.layers:
            layer.trainable = False

        # Define CNN model
        vgg_model_flatten = Sequential([
            vgg_model,
            Flatten(),  # Flatten the spatial dimensions
            Dense(tune.dense_neurons1, activation='relu'),
            Dropout(tune.dropout1)
        ])

        #25088 is the output of vff_model_flatten
        # Define LSTM model
        lstm_model = Sequential([
            LSTM(64, input_shape=(tune.sequence_of_image, tune.dense_neurons1)),
            Dropout(tune.dropout2),  # Dropout layer to prevent overfitting
            Dense(tune.dense_neurons2, activation='relu'),
            Dropout(tune.dropout3),
            Dense(1, activation='sigmoid'),
                ])

        # Combine CNN and LSTM models
        self.model = Sequential([
            TimeDistributed(vgg_model_flatten, input_shape=(tune.sequence_of_image, 224, 224, 3)),  # Apply CNN to each frame in the sequence
            (Reshape((5,tune.dense_neurons1))),
            lstm_model,
        ])
        vgg_model.summary()
        vgg_model_flatten.summary()
        self.model.summary()
        
    def train(self, train_dataset, val_dataset):
        cp = ModelCheckpoint('model_vgg_test/',monitor='val_accuracy',save_best_only=True)
            # EarlyStopping callback to stop training when validation accuracy stops improving
        es = EarlyStopping(monitor='val_accuracy', patience=30, restore_best_weights=True)
        log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
        #tensor board
        tb= tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
                # Shuffle training dataset before each epoch
        # train_dataset_shuffled = train_dataset.shuffle(buffer_size=train_dataset.cardinality(), reshuffle_each_iteration=tune.reshuffle)
        self.model.compile(loss=BinaryCrossentropy(), optimizer=Adam(learning_rate=tune.learning_rate),metrics=['accuracy'])
        self.model.fit(train_dataset,validation_data=val_dataset, epochs=tune.epochs, callbacks=[cp,es,accuracy_history])

In [115]:
class AccuracyHistory(Callback):
    def __init__(self):
        super().__init__()
        self.epoch_count = []
        self.train_accuracy = []
        self.val_accuracy = []
        self.sequence_of_image = []
        self.learning_rate = []
        self.reshuffle =  []
        self.dropout1 = []
        self.dropout2 = []
        self.dropout3 = []
        self.dropout4 = []
        self.regularization_constant = []
        self.batch_size = []
        self.dense_neurons1 =[]
        self.dense_neurons2 =[]
        self.no_of_samples = []
        self.epochs  = []
        
    def reset_dict(self):
        self.epoch_count = []
        self.train_accuracy = []
        self.val_accuracy = []
        self.sequence_of_image = []
        self.learning_rate = []
        self.reshuffle =  []
        self.dropout1 = []
        self.dropout2 = []
        self.dropout3 = []
        self.dropout4 = []
        self.regularization_constant = []
        self.batch_size = []
        self.dense_neurons1 =[]
        self.dense_neurons2 =[]
        self.no_of_samples = []
        self.epochs  = []
    def on_epoch_end(self, epoch, logs=None):
        self.epoch_count.append(epoch + 1)
        self.train_accuracy.append(logs.get('accuracy'))
        self.val_accuracy.append(logs.get('val_accuracy'))
        self.sequence_of_image.append(tune.sequence_of_image)
        self.learning_rate.append(tune.learning_rate)
        self.reshuffle.append(tune.reshuffle)
        self.dropout1.append(tune.dropout1)
        self.dropout2.append(tune.dropout2)
        self.dropout3.append(tune.dropout3)
        self.dropout4.append(tune.dropout4)
        self.regularization_constant.append(tune.regularization_constant)
        self.batch_size.append(tune.batch_size)
        self.dense_neurons1.append(tune.dense_neurons1)
        self.dense_neurons2.append(tune.dense_neurons2)
        self.no_of_samples.append(tune.no_of_samples)
        self.epochs .append(tune.epochs )
    def create_accuracy_dataframe(self):
        accuracy_df = pd.DataFrame({
            'Epoch': self.epoch_count,
            'Train_Accuracy': self.train_accuracy,
            'Val_Accuracy': self.val_accuracy,
            'Sequence_of_Image': self.sequence_of_image,
            'Learning_Rate': self.learning_rate,
            'Reshuffle': self.reshuffle,
            'Dropout1': self.dropout1,
            'Dropout2': self.dropout2,
            'Dropout3': self.dropout3,
            'Dropout4': self.dropout4,
            'Regularization_Constant': self.regularization_constant,
            'Batch_Size': self.batch_size,
            'dense_neurons1': self.dense_neurons1,
            'dense_neurons2': self.dense_neurons2,
            'no_of_samples':self.no_of_samples,
            'epochs':self.epochs 
        })
        return accuracy_df    
    def save_to_csv(self, accuracy_df):
            # Start with summary1.csv
            file_number = 0
            while True:
                filename = 'summary' + str(file_number) + '.csv'
                # Check if the file already exists
                if not os.path.isfile(filename):
                    break
                file_number += 1
            # filename = 'model' + str(file_number) + '.h5'
            # network.model.save(filename)
            accuracy_df.to_csv(filename, index=False)    
        

In [116]:
class tuning():
    def __init__(self):
        self.sequence_of_image_array = [5,6,8,9,10]
        self.learning_rate_array = [0.00005,0.00003, 0.00004, 0.00001,0.0000008, 0.000006 ]
        self.reshuffle_array=[False, True]
        self.regularization_constant_array = [0.01, 0.05, 0.1, 0.2, 0.3]
        self.dense_neurons2_array = [8, 16, 32]
        
        self.sequence_of_image =  self.sequence_of_image_array[0]
        self.learning_rate = self.learning_rate_array[1]
        self.reshuffle =  self.reshuffle_array[0]
        self.dropout1 = 0.5
        self.dropout2 = 0.5
        self.dropout3 = 0.5
        self.dropout4 = 0.5
        self.regularization_constant = 0.001
        self.batch_size = 8
        self.dense_neurons1 = 64
        self.dense_neurons2 = 8
        self.csv_id = 0
        self.no_of_samples = 50
        self.epochs = 40

        
    def start_training(self):
        try:
            manage_data2.load_data(no_of_samples=self.no_of_samples)
            manage_data2.shuffle_file_paths()
            manage_data2.create_split_filepaths()
            manage_data2.create_split_datasets()
            network.vgg_lstm()
            network.train(manage_data2.train_dataset, manage_data2.val_dataset)
        
        # Ensure accuracy data is saved even if training is interrupted 
        finally:        
            # Create a DataFrame from the accuracy history lists
            accuracy_df = accuracy_history.create_accuracy_dataframe()

            # Save the DataFrame to a CSV file
            accuracy_history.save_to_csv(accuracy_df)
            accuracy_history.reset_dict()                    
    def randomize(self):
        
        for value in self.regularization_constant_array:
            self.regularization_constant = value           
            self.start_training()
        self.regularization_constant = 0.001
            
        for value in self.learning_rate_array:
            self.learning_rate = value           
            self.start_training()
        self.learning_rate = 0.00001
        for value in self.sequence_of_image_array:
            self.sequence_of_image = value           
            self.start_training()
        self.sequence_of_image = 5

In [117]:
manage_data = Manage_data()
network = create_network()
manage_data2 = Manage_data2()
tune = tuning()
accuracy_history = AccuracyHistory()

In [118]:
# manage_data2.load_data(no_of_samples=10)
# manage_data2.shuffle_file_paths()
# manage_data2.create_split_filepaths()
# print('train_filepaths', manage_data2.train_filepaths.shape)
# print('test_filepaths', manage_data2.test_filepaths.shape)
# print('val_filepaths', manage_data2.val_filepaths.shape)


In [119]:
# manage_data2.create_split_datasets()
# for batch in manage_data2.train_dataset.take(1):  # Take one batch to print its shape
#     images_batch, labels_batch = batch
#     print("Images batch shape:", images_batch.shape)
#     print("Labels batch shape:", labels_batch.shape)
# for batch in manage_data2.test_dataset.take(1):  # Take one batch to print its shape
#     images_batch, labels_batch = batch
#     print("Images batch shape:", images_batch.shape)
#     print("Labels batch shape:", labels_batch.shape)
# for batch in manage_data2.val_dataset.take(1):  # Take one batch to print its shape
#     images_batch, labels_batch = batch
#     print("Images batch shape:", images_batch.shape)
#     print("Labels batch shape:", labels_batch.shape)


In [120]:
tune.randomize()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_15 (InputLayer)       [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

INFO:tensorflow:Assets written to: model_vgg_test/assets


68/68 [==============================] - 26s 321ms/step - loss: 0.7165 - accuracy: 0.5387 - val_loss: 0.6719 - val_accuracy: 0.6323
Epoch 2/5
67/68 [============================>.] - ETA: 0s - loss: 0.6989 - accuracy: 0.5616INFO:tensorflow:Assets written to: model_vgg_test/assets


INFO:tensorflow:Assets written to: model_vgg_test/assets


68/68 [==============================] - 21s 314ms/step - loss: 0.7030 - accuracy: 0.5590 - val_loss: 0.6454 - val_accuracy: 0.6387
Epoch 3/5
68/68 [==============================] - ETA: 0s - loss: 0.6832 - accuracy: 0.5683INFO:tensorflow:Assets written to: model_vgg_test/assets


INFO:tensorflow:Assets written to: model_vgg_test/assets


68/68 [==============================] - 21s 315ms/step - loss: 0.6832 - accuracy: 0.5683 - val_loss: 0.6418 - val_accuracy: 0.6645
Epoch 4/5
68/68 [==============================] - 16s 227ms/step - loss: 0.6818 - accuracy: 0.5517 - val_loss: 0.6797 - val_accuracy: 0.4387
Epoch 5/5
67/68 [============================>.] - ETA: 0s - loss: 0.6754 - accuracy: 0.5840

In [ ]:
# from tensorflow.keras.models import load_model

# # Load the model from the .h5 file
# model = load_model('model_vgg.h5')
# model.evaluate(manage_data2.test_dataset)

# # Print the model summary to confirm it was loaded correctly
# model.summary()